### 以下内容是重复了下面的开源项目
https://github.com/debbiemarkslab/neural-fingerprint-theano

In [1]:
import getpass
import numpy as np

from neural_fingerprint_theano.code.prediction import seqHelper
import theano
import theano.tensor as T
#from theano.tensor import *
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import lasagne
from lasagne.layers import InputLayer, MergeLayer
from lasagne import nonlinearities
from lasagne.random import get_rng

/home/xtalpi/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [34]:
data_dir = '/home/'+getpass.getuser()+'/git_test/neural_fingerprint_theano/data'
## logP, SMILES 数据
expr_filename = data_dir+'/csv_files/logSolubilityTest.csv'
## 生成的分子指纹数据，用于生成训练矩阵 X
fingerprint_filename = data_dir + '/temp/logSolubilityInput_withRDKITidx.pkl'

smiles_to_measurement,smiles_to_atom_info,smiles_to_bond_info,\
smiles_to_atom_neighbors,smiles_to_bond_neighbors,smiles_to_atom_mask,\
smiles_to_rdkit_list,max_atom_len,max_bond_len,num_atom_features,num_bond_features\
     = seqHelper.read_in_data(expr_filename,fingerprint_filename)

print 'max_atom_len: ', max_atom_len, 'max_bond_len: ', max_bond_len
print len(smiles_to_measurement), type(smiles_to_measurement)

sm = smiles_to_measurement.keys()[0]
print smiles_to_measurement[sm]
print smiles_to_atom_info[sm].shape
print smiles_to_bond_info[sm].shape
print smiles_to_atom_neighbors[sm].shape
print smiles_to_bond_neighbors[sm].shape
print smiles_to_atom_mask[sm].shape
print smiles_to_rdkit_list[sm]
print [x.shape for x in smiles_to_atom_info.values()[:10]]

max_atom_len:  56 max_bond_len:  63
1122 <type 'dict'>
-5.613
(56, 68)
(63, 6)
(56, 6)
(56, 6)
(56,)
[ 0  7 11 15 17 19 22 24 26 28  2  5  9 12 13 20 27  1  3  4  6  8 10 18 25
 14 16 21 23]
[(56, 68), (56, 68), (56, 68), (56, 68), (56, 68), (56, 68), (56, 68), (56, 68), (56, 68), (56, 68)]


### 定义符号变量

In [36]:
#define my theano variables
input_atom       = T.ftensor3('input_atom')
input_atom_index = T.itensor3('input_atom_index')
input_bonds      = T.ftensor3('input_bonds')
input_bond_index = T.itensor3('input_mask_attn')
input_mask       = T.fmatrix('input_mask_attn')
target_vals      = T.fvector('output_data')

In [39]:
def buildCNNFingerprint(input_atom, input_bonds, input_atom_index, input_bond_index, input_mask, \
    max_atom_len, max_bond_len, input_atom_dim, input_bond_dim, input_index_dim, fingerprint_dim):
    dropout_prob = 0.0
    network_vals = {}

    #take in input layers for the atom and the bonds
    l_in_atom = InputLayer(shape=(None,max_atom_len,input_atom_dim), \
        input_var=input_atom)
    l_in_bond = InputLayer(shape=(None,max_bond_len,input_bond_dim), \
        input_var=input_bonds)
    
    #take in layers for the indexing into the atoms and bonds
    l_index_atom = InputLayer(shape=(None,max_atom_len,input_index_dim), \
        input_var=input_atom_index)
    l_index_bond = InputLayer(shape=(None,max_atom_len,input_index_dim), \
        input_var=input_bond_index)
    #take in input for the mask
    l_mask = InputLayer(shape=(None,max_atom_len), input_var=input_mask)
    
    #get the number of hidden units for the first layer
    first_hidden_units_num = fingerprint_network_architecture[0]
    print input_atom_dim, input_bond_dim
    #get the embedding of the sequences we are encoding
    #network_vals['hiddens_for_atoms'] = FingerprintHiddensLayer([l_index_atom,l_index_bond,\
    #    l_in_atom,l_in_bond,l_in_atom,l_mask], input_atom_dim, input_atom_dim,
    #    input_bond_dim,first_hidden_units_num,max_atom_len)
    
    
    print l_in_atom.output_shape, l_in_bond.output_shape
    print l_index_atom.output_shape, l_index_bond.output_shape
    print l_mask.output_shape
    #print network_vals['hiddens_for_atoms'].input_shapes

In [40]:
input_index_dim = 6
output_dim = 1
fingerprint_dim = 265
batch_size = 100
final_layer_type = lasagne.nonlinearities.linear
fingerprint_network_architecture=[500]*5
neural_net=[1000]

buildCNNFingerprint(input_atom, input_bonds, \
    input_atom_index, input_bond_index, input_mask, max_atom_len, max_bond_len, num_atom_features, \
    num_bond_features, input_index_dim, fingerprint_dim)


68 6
(None, 56, 68) (None, 63, 6)
(None, 56, 6) (None, 56, 6)
(None, 56)


In [ ]:
class FingerprintHiddensLayer(MergeLayer):
    def __init__(self, incoming, input_feature_num, input_atom_num, input_bond_num,
        hidden_units_num, max_atom_len, p_dropout=0.0,\
        W_neighbors=lasagne.init.GlorotUniform(),b_neighbors=lasagne.init.Constant(0.),\
        W_atoms=lasagne.init.GlorotUniform(),b_atoms=lasagne.init.Constant(0.),\
        nonlinearity=nonlinearities.rectify,batch_normalization=True, **kwargs):
        super(FingerprintHiddensLayer, self).__init__(incoming, **kwargs)
        self.nonlinearity = (lasagne.nonlinearities.identity if nonlinearity is None
                             else nonlinearity)

        #initlialize the values of the weight matrices I'm going to use to transform
        self.W_neighbors = self.add_param(W_neighbors,(input_feature_num+input_bond_num,hidden_units_num),name="W_neighbors")
        self.b_neighbors = self.add_param(b_neighbors,(hidden_units_num,),name="b_neighbors",regularizable=False)

        self.W_atoms = self.add_param(W_atoms,(input_atom_num,hidden_units_num),name="W_atoms")
        self.b_atoms = self.add_param(b_atoms,(hidden_units_num,),name="b_atoms",regularizable=False)

        self.num_units = hidden_units_num
        self.atom_num = input_atom_num
        self.input_feature_num = input_feature_num

        self.p_dropout = p_dropout
        self._srng = RandomStreams(get_rng().randint(1, 2147462579))
        self.length = max_atom_len

In [53]:
input_index_dim = 6
output_dim = 1
fingerprint_dim = 265
batch_size = 100
final_layer_type = lasagne.nonlinearities.linear
fingerprint_network_architecture=[500]*5
neural_net=[1000]

input_atom = ftensor3('input_atom')
input_atom_index = itensor3('input_atom_index')
input_bonds = ftensor3('input_bonds')
input_bond_index = itensor3('input_mask_attn')
input_mask = fmatrix('input_mask_attn')
target_vals = fvector('output_data')
cnn_model = buildCNNFingerprint(input_atom, input_bonds, \
    input_atom_index, input_bond_index, input_mask, max_atom_len, max_bond_len, num_atom_features, \
    num_bond_features, input_index_dim, fingerprint_dim, batch_size, output_dim, final_layer_type, \
    fingerprint_network_architecture,neural_net)

(None, 56, 68) (None, 63, 6)
(None, 56, 6) (None, 56, 6)
(None, 56)
[(None, 56, 6), (None, 56, 6), (None, 56, 68), (None, 63, 6), (None, 56, 68), (None, 56)]


NotImplementedError: 